# Preprocessing et feature engineering

Etape 1 du pipeline

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

# Définition des paramètres esthétiques généraux

%matplotlib inline
sns.set_theme(style = "whitegrid", palette = "bright")

# Importation du jeu de données et enregistrement dans le DataFrame `df`

df = pd.read_csv("../data/weatherAUS.csv")

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145460 entries, 0 to 145459
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           145460 non-null  object 
 1   Location       145460 non-null  object 
 2   MinTemp        143975 non-null  float64
 3   MaxTemp        144199 non-null  float64
 4   Rainfall       142199 non-null  float64
 5   Evaporation    82670 non-null   float64
 6   Sunshine       75625 non-null   float64
 7   WindGustDir    135134 non-null  object 
 8   WindGustSpeed  135197 non-null  float64
 9   WindDir9am     134894 non-null  object 
 10  WindDir3pm     141232 non-null  object 
 11  WindSpeed9am   143693 non-null  float64
 12  WindSpeed3pm   142398 non-null  float64
 13  Humidity9am    142806 non-null  float64
 14  Humidity3pm    140953 non-null  float64
 15  Pressure9am    130395 non-null  float64
 16  Pressure3pm    130432 non-null  float64
 17  Cloud9am       89572 non-null

## 1. Transformation des variables catégorielles

In [12]:
################################################ Création des colonnes year, month, date
# Mise au format datetime de la colonne 'Date':
df['Date'] = pd.to_datetime(df['Date'])

# Extraction d'informations depuis la variable date
# Année:
df["Year"] = df['Date'].dt.year

# Mois
df["Month"] = df['Date'].dt.month

# Jour du mois
df["Day"] = df['Date'].dt.day

In [19]:
# Numérisation de RainToday 
df.dropna(subset = 'RainToday', inplace = True) # On drop ici pour réaffecter le type de la colonne à entier.
df['RainToday'].replace(to_replace = ['Yes', 'No'], value = [1, 0], inplace = True)
df['RainToday'] = df['RainToday'].astype(int)


In [20]:
# Numérisation de RainTomorrow 
df.dropna(subset = 'RainTomorrow', inplace = True) # On drop ici pour réaffecter le type de la colonne à entier.
df['RainTomorrow'].replace(to_replace = ['Yes', 'No'], value = [1, 0], inplace = True)
df['RainTomorrow'] = df['RainTomorrow'].astype(int)

In [21]:
#On passe de 145 460 entrées à 140 787. On a perdu 3.2% des données à ce stade.

In [47]:
# Encodage des noms des stations:

df['LocationNum'] = df['Location']  # création d'une copie de la colonne 'Location'

df['LocationNum'].replace(to_replace = df['Location'].unique(), value =np.arange(len(df['Location'].unique())), inplace = True ) #remplacement par le code numérique.

In [33]:
# Numérisation WindGustDir, WindDir9am, WindDir3pm

# Création d'un dictionnaire pour convertir, dans « WindGustDir », les directions du vent en radians

WindGustDir_directions_to_radians_dictionary = {
    "N": np.pi / 2,
    "E": 0.0,
    "S": 3 * np.pi,
    "W": np.pi,
    "NE": np.pi / 4,
    "SE": 7 * np.pi / 4,
    "SW": 5 * np.pi / 4,
    "NW": 3 * np.pi / 4,
    "NNE": 0.375 * np.pi,
    "ENE": 0.125 * np.pi,
    "ESE": 1.875 * np.pi,
    "SSE": 1.625 * np.pi,
    "SSW": 1.375 * np.pi,
    "WSW": 1.125 * np.pi,
    "WNW": 0.875 * np.pi,
    "NNW": 0.625 * np.pi
}

# Création des trois colonnes copies numériques:

df['WindGustDirNum'] = df['WindGustDir']
df['WindDir9amNum'] = df['WindDir9am']
df['WindDir3pmNum'] = df['WindDir3pm']

# Remplacement des catégories par les valeurs:
df['WindGustDirNum'].replace(to_replace = WindGustDir_directions_to_radians_dictionary, inplace = True)
df['WindDir9amNum'].replace(to_replace = WindGustDir_directions_to_radians_dictionary, inplace = True)
df['WindDir3pmNum'].replace(to_replace = WindGustDir_directions_to_radians_dictionary, inplace = True)


## 2. Ajout de nouvelles données

In [41]:
# Import des données gps
gps = pd.read_excel("../carto/station_gps_region.xlsx")
gps.head()

# NonMes contient la liste des grandeurs non mesurées par la station correspondante.
# Exception: 'all' devrait s'appeler 'none'.

,Location,Latitude,Longitude,CodeRegion,NonMes
0,Albury,-36.080477,146.916280,NSW,Evaporation_Sunshine
1,BadgerysCreek,-33.881667,150.744163,NSW,Evaporation_Sunshine_Cloud9am_Cloud3pm
2,Cobar,-31.498333,145.834444,NSW,all
3,CoffsHarbour,-30.296241,153.113529,NSW,all
4,Moree,-29.461720,149.840715,NSW,all


In [51]:
# Fuison de df et gps pour ajouter les 4 colonnes de gps à df, à savoir: Latitude, Longitude, CodeRegion, NonMes
df = df.merge(right = gps, on = 'Location', how = 'inner')
df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,month,day,LocationNum,WindGustDirNum,WindDir9amNum,WindDir3pmNum,Latitude,Longitude,CodeRegion,NonMes
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,12,1,0,3.141593,3.141593,2.748894,-36.080477,146.91628,NSW,Evaporation_Sunshine
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,12,2,0,2.748894,1.963495,3.534292,-36.080477,146.91628,NSW,Evaporation_Sunshine
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,12,3,0,3.534292,3.141593,3.534292,-36.080477,146.91628,NSW,Evaporation_Sunshine
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,12,4,0,0.785398,5.497787,0.000000,-36.080477,146.91628,NSW,Evaporation_Sunshine
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,12,5,0,3.141593,0.392699,2.356194,-36.080477,146.91628,NSW,Evaporation_Sunshine


In [52]:
# Encodage des noms des régions:

df['CodeRegionNum'] = df['CodeRegion']  # création d'une copie de la colonne 'Location'

df['CodeRegionNum'].replace(to_replace = df['CodeRegion'].unique(), value =np.arange(len(df['CodeRegion'].unique())), inplace = True ) #remplacement par le code numérique.

In [53]:
# Encodage des noms des valeurs non mesurées:

df['NonMesNum'] = df['NonMes']  # création d'une copie de la colonne 'Location'

df['NonMesNum'].replace(to_replace = df['NonMes'].unique(), value =np.arange(len(df['NonMes'].unique())), inplace = True ) #remplacement par le code numérique.

In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140787 entries, 0 to 140786
Data columns (total 36 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   Date            140787 non-null  datetime64[ns]
 1   Location        140787 non-null  object        
 2   MinTemp         140319 non-null  float64       
 3   MaxTemp         140480 non-null  float64       
 4   Rainfall        140787 non-null  float64       
 5   Evaporation     81093 non-null   float64       
 6   Sunshine        73982 non-null   float64       
 7   WindGustDir     131624 non-null  object        
 8   WindGustSpeed   131682 non-null  float64       
 9   WindDir9am      131127 non-null  object        
 10  WindDir3pm      137117 non-null  object        
 11  WindSpeed9am    139732 non-null  float64       
 12  WindSpeed3pm    138256 non-null  float64       
 13  Humidity9am     139270 non-null  float64       
 14  Humidity3pm     137286 non-null  flo

In [55]:
# Ecriture du csv preprocessé
df.to_csv('../data/weatherAUS_preproc.csv')

In [ ]:
# Question: faut-il déjà drop toutes les infos catégorielles ici ?
# Soit on fait ça, soit on drop avant la répartition en 9df
# Ou alors on répartit en 9df et on drop au niveau de la gestion des nan pour aboutir à un fichier prêt à la modélisation.